In [61]:
from lightgbm import LGBMRegressor
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder, RobustScaler, StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from sklearn.linear_model import Ridge, Lasso, LassoCV, LinearRegression
import time
from datetime import datetime
import matplotlib.pyplot as plt
import os
import numpy.ma as ma
from sklearn import base
import warnings
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [62]:
path_train = "data/train_data.csv"
path_test = "data/test_data.csv"
target_ = "contest-tmp2m-14d__tmp2m"

In [63]:
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

date_col = "startdate"

df_train[date_col] = pd.to_datetime(df_train[date_col])
df_test[date_col] = pd.to_datetime(df_test[date_col])

In [64]:
df_train.head()

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0,0.0,0.833333,2014-09-01,237.00,29.02,31.64,29.57,30.73,29.71,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,1,0.0,0.833333,2014-09-02,228.90,29.02,31.64,29.57,30.73,29.71,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,2,0.0,0.833333,2014-09-03,220.69,29.02,31.64,29.57,30.73,29.71,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,3,0.0,0.833333,2014-09-04,225.28,29.02,31.64,29.57,30.73,29.71,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,4,0.0,0.833333,2014-09-05,237.24,29.02,31.64,29.57,30.73,29.71,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95


# Fill null

In [65]:
def filter_na_cols(df):
    count_na_df = df.isna().sum()
    if count_na_df[count_na_df > 0].tolist():
        return count_na_df[count_na_df > 0]
    else:
        return 'Clean dataset'

col_na = [
    'nmme0-tmp2m-34w__ccsm30',
    'nmme-tmp2m-56w__ccsm3',
    'nmme-prate-34w__ccsm3',
    'nmme0-prate-56w__ccsm30',
    'nmme0-prate-34w__ccsm30',
    'nmme-prate-56w__ccsm3',
    'nmme-tmp2m-34w__ccsm3']

g_means =  ['nmme0-tmp2m-34w__nmme0mean',
 'nmme-tmp2m-56w__nmmemean',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-56w__nmme0mean',
 'nmme0-prate-34w__nmme0mean',
 'nmme-prate-56w__nmmemean',
 'nmme-tmp2m-34w__nmmemean']


g_1 = ['nmme0-tmp2m-34w__cancm30',
'nmme0-tmp2m-34w__cancm40',
'nmme0-tmp2m-34w__ccsm40',
'nmme0-tmp2m-34w__cfsv20',
'nmme0-tmp2m-34w__gfdlflora0',
'nmme0-tmp2m-34w__gfdlflorb0',
'nmme0-tmp2m-34w__gfdl0',
'nmme0-tmp2m-34w__nasa0']

g_2 = ['nmme-tmp2m-56w__cancm3',
'nmme-tmp2m-56w__cancm4',
'nmme-tmp2m-56w__ccsm4',
'nmme-tmp2m-56w__cfsv2',
'nmme-tmp2m-56w__gfdl',
'nmme-tmp2m-56w__gfdlflora',
'nmme-tmp2m-56w__gfdlflorb',
'nmme-tmp2m-56w__nasa']

g_3 = ['nmme-prate-34w__cancm3',
'nmme-prate-34w__cancm4',
'nmme-prate-34w__ccsm4',
'nmme-prate-34w__cfsv2',
'nmme-prate-34w__gfdl',
'nmme-prate-34w__gfdlflora',
'nmme-prate-34w__gfdlflorb',
'nmme-prate-34w__nasa']

g_4 = [ 'nmme0-prate-56w__cancm30',
'nmme0-prate-56w__cancm40',
'nmme0-prate-56w__ccsm40',
'nmme0-prate-56w__cfsv20',
'nmme0-prate-56w__gfdlflora0',
'nmme0-prate-56w__gfdlflorb0',
'nmme0-prate-56w__gfdl0',
'nmme0-prate-56w__nasa0']

g_5 = ['nmme0-prate-34w__cancm30',
'nmme0-prate-34w__cancm40',
'nmme0-prate-34w__ccsm40',
'nmme0-prate-34w__cfsv20',
'nmme0-prate-34w__gfdlflora0',
'nmme0-prate-34w__gfdlflorb0',
'nmme0-prate-34w__gfdl0',
'nmme0-prate-34w__nasa0']

g_6 = ['nmme-prate-56w__cancm3',
'nmme-prate-56w__cancm4',
'nmme-prate-56w__ccsm4',
'nmme-prate-56w__cfsv2',
'nmme-prate-56w__gfdl',
'nmme-prate-56w__gfdlflora',
'nmme-prate-56w__gfdlflorb',
'nmme-prate-56w__nasa']

g_7 = ['nmme-tmp2m-34w__cancm3',
'nmme-tmp2m-34w__cancm4',
'nmme-tmp2m-34w__ccsm4',
'nmme-tmp2m-34w__cfsv2',
'nmme-tmp2m-34w__gfdl',
'nmme-tmp2m-34w__gfdlflora',
'nmme-tmp2m-34w__gfdlflorb',
'nmme-tmp2m-34w__nasa']

gs = [g_1, g_2, g_3, g_4, g_5, g_6, g_7]

zip_cols = zip(col_na, gs, g_means)
for c, g, m in tqdm(zip_cols):
    df_train[c] = (df_train[m]*9) - df_train[g].sum(1)

7it [00:03,  1.92it/s]


In [66]:
df_train.drop(columns='ccsm30', inplace=True)
df_test.drop(columns='ccsm30', inplace=True)

# Feature engineering

In [67]:
target_ = 'contest-tmp2m-14d__tmp2m'

col = [each for each in df_train.columns if "contest" in each and each != target_]

def get_idx(lat, lon):
    return str(round(lat, 4)) + "_" + str(round(lon, 4))

df_train['idx'] = np.vectorize(get_idx)(df_train['lat'], df_train['lon'])
df_test['idx'] = np.vectorize(get_idx)(df_test['lat'], df_test['lon'])

for each in col:
    df_train[each + "_lag_1"] = df_train.groupby("idx")[each].shift(1).bfill()
    df_test[each + "_lag_1"] = df_test.groupby("idx")[each].shift(1).bfill()
    df_train[each + "_shift_1"] = df_train.groupby("idx")[each].shift(-1).ffill()
    df_test[each + "_shitf_1"] = df_test.groupby("idx")[each].shift(-1).ffill()

In [68]:
bin_col = ['nmme-tmp2m-34w__nmmemean', 'nmme-tmp2m-56w__nmmemean', 'nmme0-tmp2m-34w__nmme0mean', 'nmme0mean']

def bin_feature_tpm2m(x):
    if x < -5:
        return 'A'
    elif x < 0:
        return 'B'
    elif x < 5:
        return 'C'
    elif x < 10:
        return 'D'
    elif x < 15:
        return 'E'
    elif x < 20:
        return 'F'
    elif x < 25:
        return 'G'
    else:
        return 'H'

for each in bin_col:
    df_train[each + "_bin"] = np.vectorize(bin_feature_tpm2m)(df_train[each])
    df_test[each + "_bin"] = np.vectorize(bin_feature_tpm2m)(df_test[each])

In [69]:
imp_contest_cols = [
    'contest-pevpr-sfc-gauss-14d__pevpr',
    'contest-pres-sfc-gauss-14d__pres',
    'contest-prwtr-eatm-14d__prwtr',
    'contest-slp-14d__slp',
    'contest-wind-h10-14d__wind-hgt-10',
    'contest-wind-h100-14d__wind-hgt-100',
    'contest-wind-h500-14d__wind-hgt-500',
    'contest-wind-uwnd-250-14d__wind-uwnd-250',
    'contest-wind-vwnd-925-14d__wind-vwnd-925'
]

def get_name_cols(suffix=''):
    name_cols = [col + f'{suffix}' for col in imp_contest_cols]

    return name_cols

def feature_engineer(df, cols):

    # df.loc[:, 'idx'] = np.vectorize(get_idx)(df['lat'], df['lon'])
    df.loc[:, 'month'] = df['startdate'].dt.month
    df.loc[:, 'year']  = df['startdate'].dt.year

    mean_contest_cols = df.groupby(['idx', 'month', 'year'])[cols].transform(lambda x: x.mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_mean'))))
    std_contest_cols = df.groupby(['idx', 'month', 'year'])[cols].transform(lambda x: x.std()) \
        .rename(columns=dict(zip(cols, get_name_cols('_std'))))
    rolling_mean_contest_cols = df.groupby('idx')[cols].transform(lambda x: x.rolling(window=7, min_periods=3, win_type="triang").mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_rolling_mean'))))
    rolling_std_contest_cols = df.groupby('idx')[cols].transform(lambda x: x.rolling(window=7, min_periods=3).std()) \
        .rename(columns=dict(zip(cols, get_name_cols('_rolling_std'))))
    expanding_mean_contest_cols = df.groupby('idx')[cols].transform(lambda x: x.expanding(2).mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_expanding_mean'))))
    expanding_std_contest_cols = df.groupby('idx')[cols].transform(lambda x: x.expanding(2).std()) \
        .rename(columns=dict(zip(cols, get_name_cols('_expanding_std'))))
    exponentially_weighted_average = df.groupby('idx')[cols].transform(lambda x: x.shift(1).ewm(alpha=0.95).mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_ewa'))))

    df = pd.concat([df, mean_contest_cols, std_contest_cols, rolling_mean_contest_cols,
                    rolling_std_contest_cols, expanding_mean_contest_cols,
                    expanding_std_contest_cols, exponentially_weighted_average], axis=1)

    df = df.fillna(method='bfill')
    df.drop(columns=['month', 'year'], inplace=True)

    return df

df_train = feature_engineer(df_train, imp_contest_cols)
df_test = feature_engineer(df_test, imp_contest_cols)

In [70]:
df_train = df_train.drop(columns=["idx"])
df_test = df_test.drop(columns=["idx"])

In [71]:
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

def fourier_series(dates, period, series_order):
    t = np.array(
        (dates - datetime(1970, 1, 1))
            .dt.total_seconds()
            .astype(float)
    ) / (3600 * 24.)

    return np.column_stack([
        fun((2.0 * (i + 1) * np.pi * t / period))
        for i in range(series_order)
        for fun in (np.sin, np.cos)
    ])

def make_seasonality_features(dates, period, series_order, prefix):
    features = fourier_series(dates, period, series_order)
    columns = [
        '{}_delim_{}'.format(prefix, i + 1)
        for i in range(features.shape[1])
    ]
    return pd.DataFrame(features, columns=columns)

def fourier_transform(df):
    def reconvert(df):
        if not os.path.exists('delete_later'):
            os.mkdir('delete_later')

        df.to_csv('delete_later/df.csv')
        df = pd.read_csv('delete_later/df.csv')

        return df

    if not os.path.exists('fourier'):
        os.mkdir('fourier')

    seasonalities = {
        'monthly': {
            'period': 30.5,
            'fourier_order': 8,
        }, 'quarterly': {
            'period': 90,
            'fourier_order': 10
        }
    }

    for name, props in seasonalities.items():

        df['startdate']  = pd.to_datetime(df['startdate'])

        t = np.array(
            (df['startdate'] - datetime(1970, 1, 1))
                .dt.total_seconds()
                .astype(float)
        ) / (3600 * 24.)

        series_order = props['fourier_order']
        period = props['period']

        s = np.column_stack([
                fun((2.0 * (i + 1) * np.pi * t / period))
                for i in range(series_order)
                for fun in (np.sin, np.cos)
            ])

        columns = [
            '{}_delim_{}'.format(name, i + 1)
            for i in range(s.shape[1])
        ]

        features = pd.DataFrame(s, columns=columns)

        df = pd.concat([reconvert(df), reconvert(features)], axis=1)

    return df

def handle_idx_date(df, column_date, idx_name):
    def get_idx(lat, lon):
        return str(round(lat, 4)) + "_" + str(round(lon, 4))

    df.loc[:, idx_name] = np.vectorize(get_idx)(df['lat'], df['lon'])
    df.loc[:, column_date] = pd.to_datetime(df[column_date])
    df.loc[:, 'month'] = df[column_date].dt.month
    df.loc[:, 'year'] = df[column_date].dt.year

    df = fourier_transform(df)

    return df

def handle_feature_train_data(df, column_date="startdate", columns_cat = [], idx_name="idx"):
    df = handle_idx_date(df, column_date, idx_name)
    df = df.drop(columns = [column_date])
    columns_cat.append(idx_name)
    list_lbEncoder = []

    for each in columns_cat:
        lbE = LabelEncoder().fit(df[each])
        df[each] = lbE.transform(df[each])
        list_lbEncoder.append(lbE)

    df[columns_cat] = df[columns_cat].astype("category")

    return df, list_lbEncoder

def handle_feature_test_data(df, lbEncoder, column_date="startdate", columns_cat = [], idx_name="idx"):
    df = handle_idx_date(df, column_date, idx_name)
    df = df.drop(columns = [column_date])
    columns_cat.append(idx_name)
    list_lbEncoder = []

    for index, each in enumerate(columns_cat):
        df[each] = lbEncoder[index].transform(df[each])

    df[columns_cat] = df[columns_cat].astype("category")

    return df

In [72]:
def reconvert(df):
    if not os.path.exists('delete_later'):
        os.mkdir('delete_later')

    df.to_csv('delete_later/df.csv')
    df = pd.read_csv('delete_later/df.csv')

    return df

In [73]:
def calculate_corr_target(X_train, X_val):
    data_col = []
    data_corr_train = []
    data_corr_val = []

    for col in X_train.columns:
        corr_train = X_train['contest-tmp2m-14d__tmp2m'].corr(X_train[col])
        corr_val = X_val['contest-tmp2m-14d__tmp2m'].corr(X_val[col])

        data_col.append(col)
        data_corr_train.append(corr_train)
        data_corr_val.append(corr_val)

    corr = pd.DataFrame(data={'col': data_col, 'corr_train': data_corr_train, 'corr_val': data_corr_val})

    return corr

In [74]:
corr_df = pd.read_csv("data/correlations_with_target.csv") \
            .drop(columns='Unnamed: 0')
corr_df = corr_df[(corr_df['corr'] >= 0.7)  | (corr_df['corr'] <= -0.7)]
drop_col = corr_df["col"].values
drop_col = [each for each in drop_col if "contest" not in each and "wind" not in each and each != 'ccsm30']

# Model

## LGBM

In [81]:
def bayes_tunning():
    def CB_opt(max_depth, num_leaves, colsample_bytree, reg_alpha,
               reg_lambda, subsample, feature_fraction_bynode):

        lgb = LGBMRegressor(
            max_depth = round(max_depth),
            num_leaves=round(num_leaves),
            colsample_bytree=colsample_bytree,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            subsample=subsample,
            feature_fraction_bynode=feature_fraction_bynode,
            verbose=0,
            n_estimators = 7999,
            metric="rmse",
            # boosting_type="dart",
            data_sample_strategy='goss',
            learning_rate=0.05,
            n_jobs=-1,
            extra_trees=True
        )

        lgb.fit(X_train, y_train, eval_metric="rmse",
                categorical_feature=[
                    'climateregions__climateregion', 'idx', 'mjo1d__phase',
                    'nmme-tmp2m-34w__nmmemean_bin', 'nmme-tmp2m-56w__nmmemean_bin',
                    'nmme0-tmp2m-34w__nmme0mean_bin', 'nmme0mean_bin'
                ]
                )
        ypred_valid = lgb.predict(X_valid)

        return 1 / mean_squared_error(y_valid, ypred_valid, squared=False)

    pbounds = {
        'max_depth': (8, 17),
        'num_leaves': (15, 35),
        'colsample_bytree': (0.5, 0.8),
        'reg_alpha': (0.1, 3),
        'reg_lambda': (0.1, 3),
        # 'min_split_gain': (0.001, 0.3),
        'subsample': (0.5, 0.85),
        'feature_fraction_bynode': (0.6, 0.85)
    }

    if not os.path.exists('Params_tunning/Log'):
        os.makedirs('Params_tunning/Log')

    optimizer = BayesianOptimization(f = CB_opt, pbounds = pbounds, verbose = 2, random_state = 209)
    logger = JSONLogger(path="Params_tunning/Log/logs_{}.json".format(index))
    optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
    optimizer.maximize(init_points = 5, n_iter = 15)

    print(optimizer.max['target'])

    max_bo_params = optimizer.max['params']

    return max_bo_params

In [82]:
def get_bo_result(max_bo_params):
    opt_params = {
        'max_depth': round(max_bo_params['max_depth']),
        'num_leaves': round(max_bo_params['num_leaves']),
        'colsample_bytree': max_bo_params['colsample_bytree'],
        'reg_alpha': max_bo_params['reg_alpha'],
        'reg_lambda': max_bo_params['reg_lambda'],
        'subsample': max_bo_params['subsample'],
        'feature_fraction_bynode': max_bo_params['feature_fraction_bynode'],
        'verbose': 0,
        'n_estimators': 7999,
        'metric': 'rmse',
        # 'boosting_type': 'dart',
        'data_sample_strategy': 'goss',
        'learning_rate': 0.05,
        'n_jobs': -1,
        'extra_trees': True
    }

    return opt_params

def get_best_params(path):
    tmp = pd.read_json(path, lines=True)
    max_t = 0
    max_i = 0

    for i in range(tmp.shape[0]):
        line = tmp.loc[i, :]
        t = line['target']

        if t > max_t:
            max_t = t
            max_i = i

    max_line = tmp.loc[max_i, :]
    max_param = max_line['params']
    max_param['max_depth'] = round(max_param['max_depth'])
    max_param['num_leaves'] = round(max_param['num_leaves'])

    return max_param

In [87]:
dict_result = {}

year_valid = [2015, 2015, 2015, 2015, 2015, 2015, 2016, 2016, 2016, 2016]
month_valid = [1, 3, 5, 7, 9, 11, 1, 3, 5, 7]

# year_valid = [2015]
# month_valid = [9]

for index in range(len(year_valid)):
    t = time.time()
    y, m_s, m_e = year_valid[index], month_valid[index], month_valid[index] + 1

    print("Training - {}".format(index))
    print(f'Test on month {m_s} and {m_e}, year {y}')

    if index not in dict_result:

        print('Pre-processing...')

        X_valid = df_train[(df_train[date_col].dt.year == y) & (df_train[date_col].dt.month >= m_s) & (df_train[date_col].dt.month <= m_e)]
        X_train = df_train[~((df_train[date_col].dt.year == y) & (df_train[date_col].dt.month >= m_s) & (df_train[date_col].dt.month <= m_e))]

        y_train = X_train[target_]
        y_valid = X_valid[target_]

        date = np.array(X_valid['startdate'])

        test_index = df_test["index"].values
        X_test = df_test.copy()

        cat_cols = [i for i in X_train.select_dtypes(include='object').columns if i != date_col]
        X_train, listEncoder = handle_feature_train_data(X_train, date_col, cat_cols.copy())
        X_valid = handle_feature_test_data(X_valid, listEncoder, columns_cat=cat_cols.copy())
        X_test = handle_feature_test_data(X_test, listEncoder, columns_cat=cat_cols.copy())

        drop_ = [*drop_col, *[col for col in X_train.columns if 'Unnamed' in col]]
        # drop_ = [col for col in X_train.columns if 'Unnamed' in col]
        X_train = X_train.drop(columns=drop_)
        X_valid = X_valid.drop(columns=drop_)
        X_test = X_test.drop(columns=drop_)

        X_train = X_train.drop(columns=target_)
        X_valid = X_valid.drop(columns=target_)

        # Model
        print('Tuning...')

        # max_bo_params = bayes_tunning()
        # opt_params = get_bo_result(max_bo_params)

        opt_params = {
            'max_depth': 10,
            'num_leaves': 22,
            'colsample_bytree': 0.5319125135421559,
            'reg_alpha': 3.0,
            'reg_lambda': 0.17764163903931635,
            'subsample': 0.5531922328107065,
            'feature_fraction_bynode': 0.8438957253353605,
            'verbose': 0,
            'n_estimators': 7999,
            'metric': 'rmse',
            'data_sample_strategy': 'goss',
            'learning_rate': 0.05,
            'n_jobs': -1,
            'extra_trees': True
        }

        reg = LGBMRegressor(**opt_params)

        reg.fit(X_train, y_train,
                eval_metric='rmse',
                categorical_feature=[
                    'climateregions__climateregion', 'idx', 'mjo1d__phase',
                    'nmme-tmp2m-34w__nmmemean_bin', 'nmme-tmp2m-56w__nmmemean_bin',
                    'nmme0-tmp2m-34w__nmme0mean_bin', 'nmme0mean_bin'
                ])

        ypred_train = reg.predict(X_train)
        ypred_valid = reg.predict(X_valid)

        result_train = mean_squared_error(y_train, ypred_train, squared=False)
        result_valid = mean_squared_error(y_valid, ypred_valid, squared=False)

        print("Take time: ", time.time() - t)
        print("Train_score: {}  Valid_score: {}".format(result_train, result_valid))

        ypred_test = reg.predict(X_test)
        dict_result[index] = ypred_test

    else:
        print('Fold was trained!')

    print("-------------")

Training - 0
Test on month 1 and 2, year 2015
Pre-processing...
Tuning...
[LightGBM] [Warning] Unknown parameter: data_sample_strategy
[LightGBM] [Warning] Unknown parameter: data_sample_strategy
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030295 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Take time:  505.28909397125244
Train_score: 0.20073631327617228  Valid_score: 1.7713203020446229
-------------
Training - 1
Test on month 3 and 4, year 2015
Pre-processing...
Tuning...
[LightGBM] [Warning] Unknown parameter: data_sample_strategy
[LightGBM] [Warning] Unknown parameter: data_sample_strategy
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029195 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Take time:  481.5390908718109
Train_sco

In [88]:
ypred_test = np.mean([value for key, value in dict_result.items()], axis=0)

In [89]:
pd.DataFrame(data = {"{}".format(target_): ypred_test, "index": test_index}).to_csv("submit.csv", index=False)

In [90]:
importances = reg.feature_importances_

data = {'col': X_train.columns, 'imp': importances}

ft_imp_df = pd.DataFrame(data)
ft_imp_df.sort_values('imp', ascending=False)

,col,imp
299,idx,23632
232,nmme-tmp2m-34w__nmmemean_bin,5959
233,nmme-tmp2m-56w__nmmemean_bin,5884
99,mjo1d__phase,4889
57,climateregions__climateregion,4669
...,...,...
313,monthly_delim_12,148
102,mei__meirank,146
317,monthly_delim_16,112
301,year,1


In [91]:
ft_imp_df.to_csv('ft_imp.csv')

## Linear Regression

In [9]:
class TargetEncoder(base.BaseEstimator,base.TransformerMixin):
    def __init__(self, colnames, targetName, colSupport=None,
                  discardOriginal_col=False):
        self.colnames = colnames
        self.colSupport = colSupport
        self.targetName = targetName
        self.discardOriginal_col = discardOriginal_col
        self.new_cols = colnames + '_tar'
        self.data_transform = None

    def fit(self, X, y=None):
        if self.colSupport != None:
            self.data_transform = X.groupby([self.colnames, self.colSupport]).agg({self.targetName: 'mean'}).rename(columns={self.targetName: self.new_cols})
        else:
            self.data_transform = X.groupby([self.colnames]).agg({self.targetName: 'mean'}).rename(columns={self.targetName: self.new_cols})
        return self

    def transform(self, X,train=True):
        X = X.merge(self.data_transform.reset_index())
        encoded_feature = X[self.new_cols].values
        if train:
            print('Correlation between the new feature, {} and, {} is {}.'.format(self.new_cols,self.targetName,
                   ma.corrcoef(ma.masked_invalid(X[self.targetName].values), ma.masked_invalid(encoded_feature))[0][1]))
        if self.discardOriginal_col:
            X = X.drop(self.targetName, axis=1)
        return X

In [15]:
dict_result = {}
year_valid = [2015, 2015, 2015, 2015, 2015, 2015, 2016, 2016, 2016, 2016]
month_valid = [1, 3, 5, 7, 9, 11, 1, 3, 5, 7]
gap = 1

for index in range(len(year_valid)):
    if index in dict_result:
        continue
    y, m_s, m_e = year_valid[index], month_valid[index], month_valid[index] + 1
    X_valid = df_train[(df_train[date_col].dt.year == y) & (df_train[date_col].dt.month >= m_s) & (df_train[date_col].dt.month <= m_e)]
    X_train = df_train[~((df_train[date_col].dt.year == y) & (df_train[date_col].dt.month >= m_s) & (df_train[date_col].dt.month <= m_e))]

    # y_train = X_train[target_].copy()
    X_train = X_train.drop(columns=["index"])
    # y_valid = X_valid[target_].copy()
    X_valid = X_valid.drop(columns=["index"])

    test_index = df_test["index"].values
    X_test = df_test.drop(columns=["index"]).copy()

    cat_cols = [i for i in X_train.select_dtypes(include='object').columns if i != date_col]
    X_train, listEncoder, df_mean = handle_feature_train_data(X_train, date_col, cat_cols.copy())
    X_valid = handle_feature_test_data(X_valid, listEncoder, df_mean, columns_cat=cat_cols.copy())
    X_test = handle_feature_test_data(X_test, listEncoder, df_mean, columns_cat=cat_cols.copy())

    target_climate = TargetEncoder(colnames='climateregions__climateregion', targetName=target_).fit(X_train)
    target_idx = TargetEncoder(colnames='idx', targetName=target_).fit(X_train)

    X_train = target_climate.transform(X_train)
    X_valid = target_climate.transform(X_valid)
    X_test = target_climate.transform(X_test, train=False)
    X_train = target_idx.transform(X_train)
    X_valid = target_idx.transform(X_valid)
    X_test = target_idx.transform(X_test, train=False)

    y_train = X_train[target_].copy()
    y_valid = X_valid[target_].copy()
    X_train = X_train.drop(columns=[target_])
    X_valid = X_valid.drop(columns=[target_])

    drop_ = [*drop_col, *["mei__nip", "idx", "climateregions__climateregion", 'month', 'year'], *[col for col in X_train.columns if 'Unnamed' in col]]
    # drop_ = ["month", "day_of_year", "day_of_year_sin", "day_of_year_cos", "month_sin", "month_cos", "mei__nip"]

    X_train_ = X_train.drop(columns=drop_)
    X_valid_ = X_valid.drop(columns=drop_)
    X_test_ = X_test.drop(columns=drop_)

    print("Training - {}".format(index))

    t = time.time()
    lgb = LinearRegression(normalize=True)
    lgb.fit(X_train_, y_train)
    print("Take time: ", time.time() - t)

    ypred_test = lgb.predict(X_test_)
    dict_result[index] = ypred_test

    result_valid_ = mean_squared_error(y_valid, lgb.predict(X_valid_), squared=False)
    print(result_valid_)
    print('-----------------')

Correlation between the new feature, climateregions__climateregion_tar and, contest-tmp2m-14d__tmp2m is 0.4113783360673971.
Correlation between the new feature, climateregions__climateregion_tar and, contest-tmp2m-14d__tmp2m is 0.6052981886481146.
Correlation between the new feature, idx_tar and, contest-tmp2m-14d__tmp2m is 0.4993773015499702.
Correlation between the new feature, idx_tar and, contest-tmp2m-14d__tmp2m is 0.7619141098406116.
Training - 0
Take time:  4.278538703918457
1.7718674135449102
-----------------
Correlation between the new feature, climateregions__climateregion_tar and, contest-tmp2m-14d__tmp2m is 0.3907415511513486.
Correlation between the new feature, climateregions__climateregion_tar and, contest-tmp2m-14d__tmp2m is 0.753669979390729.
Correlation between the new feature, idx_tar and, contest-tmp2m-14d__tmp2m is 0.47645630723256754.
Correlation between the new feature, idx_tar and, contest-tmp2m-14d__tmp2m is 0.906997989695582.
Training - 1
Take time:  4.512842

## Ridge

In [26]:
dict_result1 = {}

year_valid = [2015, 2015, 2015, 2015, 2015, 2015, 2016, 2016, 2016, 2016]
month_valid = [1, 3, 5, 7, 9, 11, 1, 3, 5, 7]

for index in range(len(year_valid)):
    t = time.time()

    if index in dict_result1:
        continue
    y, m_s, m_e = year_valid[index], month_valid[index], month_valid[index] + 1

    print("Training - {}".format(index))
    print(f'Test on month {m_s} and {m_e}, year {y}')

    X_valid = df_train[(df_train[date_col].dt.year == y) & (df_train[date_col].dt.month >= m_s) & (df_train[date_col].dt.month <= m_e)]
    X_train = df_train[~((df_train[date_col].dt.year == y) & (df_train[date_col].dt.month >= m_s) & (df_train[date_col].dt.month <= m_e))]

    y_train = X_train[target_]
    y_valid = X_valid[target_]

    date = np.array(X_valid['startdate'])

    test_index = df_test["index"].values
    X_test = df_test.copy()

    cat_cols = [i for i in X_train.select_dtypes(include='object').columns if i != date_col]
    X_train, listEncoder, df_mean = handle_feature_train_data(X_train, date_col, cat_cols.copy())
    X_valid = handle_feature_test_data(X_valid, listEncoder, df_mean, columns_cat=cat_cols.copy())
    X_test = handle_feature_test_data(X_test, listEncoder, df_mean, columns_cat=cat_cols.copy())

    drop_ = [*drop_col, *[col for col in X_train.columns if 'Unnamed' in col]]
    # drop_ = [col for col in X_train.columns if 'Unnamed' in col]
    X_train = X_train.drop(columns=drop_)
    X_valid = X_valid.drop(columns=drop_)
    X_test = X_test.drop(columns=drop_)

    X_train = X_train.drop(columns=target_)
    X_valid = X_valid.drop(columns=target_)

    reg1 = Ridge(alpha=1)
    reg1.fit(X_train, y_train)

    ypred_train1 = reg1.predict(X_train)
    ypred_valid1 = reg1.predict(X_valid)

    result_train = mean_squared_error(y_train, ypred_train1, squared=False)
    result_valid = mean_squared_error(y_valid, ypred_valid1, squared=False)

    ypred_test1 = reg1.predict(X_test)
    dict_result1[index] = ypred_test1

    print("Take time: ", time.time() - t)
    print("Train_score: {}  Valid_score: {}".format(result_train, result_valid))
    print("-------------")

Training - 0
Test on month 1 and 2, year 2015
Take time:  67.92599511146545
Train_score: 1.659054119970461  Valid_score: 1.9809388998046307
-------------
Training - 1
Test on month 3 and 4, year 2015
Take time:  66.92680883407593
Train_score: 1.6772634579009948  Valid_score: 2.0286455659914506
-------------
Training - 2
Test on month 5 and 6, year 2015
Take time:  68.67658376693726
Train_score: 1.6612086151327785  Valid_score: 2.269554590719064
-------------
Training - 3
Test on month 7 and 8, year 2015
Take time:  70.87002182006836
Train_score: 1.665848919346657  Valid_score: 1.8184317217754429
-------------
Training - 4
Test on month 9 and 10, year 2015
Take time:  74.12542819976807
Train_score: 1.6797299656952385  Valid_score: 2.3728293618074683
-------------
Training - 5
Test on month 11 and 12, year 2015
Take time:  68.98904228210449
Train_score: 1.6677255381665197  Valid_score: 2.536972123021464
-------------
Training - 6
Test on month 1 and 2, year 2016
Take time:  67.1765449047

In [27]:
ypred_test = np.mean([value for key, value in dict_result.items()], axis=0)
pd.DataFrame(data = {"{}".format(target_): ypred_test, "index": test_index}).to_csv("submit_ridge_drop_cols.csv", index=False)

# Ensemble

In [29]:
def get_idx(lat, lon):
        return str(round(lat, 4)) + "_" + str(round(lon, 4))

df_train.loc[:, 'idx'] = np.vectorize(get_idx)(df_train['lat'], df_train['lon'])

In [30]:
df_test.loc[:, 'idx'] = np.vectorize(get_idx)(df_test['lat'], df_test['lon'])
df_test.loc[:, 'month'] = df_test['startdate'].dt.month
df_test.loc[:, 'month'] = df_test['startdate'].dt.month

In [33]:
idx_temp = df_test.loc[(df_test['lat'] > 0.95) & (df_test['startdate'].between('2022-12-04', '2022-12-10'))].index.tolist()

In [34]:
len(idx_temp)

364

In [35]:
df1 = pd.read_csv('submit_khiempm.csv')

df1.loc[idx_temp, target_] = df1.loc[idx_temp, target_] * 0.8

In [36]:
df1.to_csv('submit_4fun.csv', index=False)

In [113]:
idx_temp = df_test.sort_values('nmme-tmp2m-56w__cfsv2', ascending=True).head(1000).index.tolist()

In [114]:
df1.loc[idx_temp, target_] = df1.loc[idx_temp, target_] - 0.822222222222

# Error analysis

In [103]:
df1 = pd.read_csv('x_valid_drop_cols.csv')
df2 = pd.read_csv('x_valid_no_drop_cols.csv')

In [104]:
df1 = df1[(df1['lon'] >= 0.6) & (df1['lat'] > 0.8)]
df2 = df2[(df2['lon'] >= 0.6) & (df2['lat'] > 0.8)]

In [105]:
tmp1 = df1.loc[(df1['y_val'] <= -1) & (df1['month'] == 2), ['y_val', 'ypred_val_ridge', 'ypred_val_lgbm', 'startdate']]
print(mean_squared_error(tmp1['y_val'], tmp1['ypred_val_ridge'], squared=False))
print(mean_squared_error(tmp1['y_val'], tmp1['ypred_val_lgbm'], squared=False))

1.9363381564470104
3.4294402237548196


In [106]:
tmp2 = df2.loc[(df2['y_val'] <= -1) & (df2['month'] == 2), ['y_val', 'ypred_val_ridge', 'ypred_val_lgbm', 'startdate']]
print(mean_squared_error(tmp2['y_val'], tmp2['ypred_val_ridge'], squared=False))
print(mean_squared_error(tmp2['y_val'], tmp2['ypred_val_lgbm'], squared=False))

1.7393927786934862
2.974922728859122


In [107]:
tmp1

,y_val,ypred_val_ridge,ypred_val_lgbm,startdate
23463,-1.430358,-5.712171,-3.358557,2015-02-10
23464,-1.485963,-5.825051,-3.857963,2015-02-11
23465,-1.779976,-5.409787,-2.770169,2015-02-12
23466,-3.372534,-6.399891,-3.237806,2015-02-13
23467,-4.852113,-7.541411,-5.081068,2015-02-14
...,...,...,...,...
30321,-11.885797,-8.907096,-12.041913,2015-02-24
30322,-10.901809,-8.126824,-11.461912,2015-02-25
30323,-9.256792,-7.149840,-11.171508,2015-02-26
30324,-7.472682,-5.223165,-9.470506,2015-02-27


In [108]:
tmp2

,y_val,ypred_val_ridge,ypred_val_lgbm,startdate
23463,-1.430358,-6.754530,-2.863596,2015-02-10
23464,-1.485963,-6.762077,-3.105577,2015-02-11
23465,-1.779976,-6.271617,-2.310753,2015-02-12
23466,-3.372534,-7.012811,-2.988576,2015-02-13
23467,-4.852113,-7.923057,-4.437363,2015-02-14
...,...,...,...,...
30321,-11.885797,-10.774110,-11.710988,2015-02-24
30322,-10.901809,-9.936916,-11.439990,2015-02-25
30323,-9.256792,-8.952402,-10.547538,2015-02-26
30324,-7.472682,-7.280416,-9.146467,2015-02-27


In [109]:
import plotly.express as px

In [112]:
tmp1_t = tmp2.melt(id_vars='startdate', value_vars=['y_val', 'ypred_val_ridge', 'ypred_val_lgbm']).reset_index()
px.line(tmp1_t, 'index', 'value', color='variable')

In [111]:
tmp1_t

,startdate,variable,value
0,2015-02-10,y_val,-1.430358
1,2015-02-11,y_val,-1.485963
2,2015-02-12,y_val,-1.779976
3,2015-02-13,y_val,-3.372534
4,2015-02-14,y_val,-4.852113
...,...,...,...
3802,2015-02-24,ypred_val_lgbm,-11.710988
3803,2015-02-25,ypred_val_lgbm,-11.439990
3804,2015-02-26,ypred_val_lgbm,-10.547538
3805,2015-02-27,ypred_val_lgbm,-9.146467
